# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on Data Joins and JSON columns** </center>
---
**Profesor**: Pablo Camarillo Ramirez

# Create SparkSession

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on data joins") \
    .master("spark://9eae4bedccb3:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/24 01:11:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Manipulating JSON Columns

In [2]:
from pcamarillor.spark_utils import SparkUtils
json_schema = SparkUtils.generate_schema([("id", "int"), ("json_col", "string")])
json_data = [
    (1, '{"name": "Alice", "age": 25, "payments": [34, 433, 54], "address": {"city": "New York", "zip": "10001"}}'),
    (2, '{"name": "Bob", "age": 30, "address": {"city": "Los Angeles", "zip": "90001"}}'),
    (3, '{"name": "Charlie", "age": 35, "address": {"city": "Chicago", "zip": "60601"}}')
]

df_json = spark.createDataFrame(json_data, json_schema)
df_json.show(truncate=False)


+---+--------------------------------------------------------------------------------------------------------+
|id |json_col                                                                                                |
+---+--------------------------------------------------------------------------------------------------------+
|1  |{"name": "Alice", "age": 25, "payments": [34, 433, 54], "address": {"city": "New York", "zip": "10001"}}|
|2  |{"name": "Bob", "age": 30, "address": {"city": "Los Angeles", "zip": "90001"}}                          |
|3  |{"name": "Charlie", "age": 35, "address": {"city": "Chicago", "zip": "60601"}}                          |
+---+--------------------------------------------------------------------------------------------------------+



### Extract a JSON column with get_json_object function

In [3]:
from pyspark.sql.functions import get_json_object
df_json = df_json.withColumn("name", get_json_object(df_json.json_col, "$.name"))
df_json.show()

+---+--------------------+-------+
| id|            json_col|   name|
+---+--------------------+-------+
|  1|{"name": "Alice",...|  Alice|
|  2|{"name": "Bob", "...|    Bob|
|  3|{"name": "Charlie...|Charlie|
+---+--------------------+-------+



In [4]:
df_json = df_json.withColumn("age", get_json_object(df_json.json_col, "$.age"))
df_json.show()

+---+--------------------+-------+---+
| id|            json_col|   name|age|
+---+--------------------+-------+---+
|  1|{"name": "Alice",...|  Alice| 25|
|  2|{"name": "Bob", "...|    Bob| 30|
|  3|{"name": "Charlie...|Charlie| 35|
+---+--------------------+-------+---+



### Extact a JSON column with from_json

In [ ]:
from pyspark.sql.functions import from_json
# Deine the schema of the JSON object
people_schema = SparkUtils.generate_schema([("name", "string"),
                                            ("age", "int"),
                                            ("payments", "array_int"),
                                            ("address", "struct")])
df_parsed = df_json.withColumn("parsed", from_json(df_json.json_col, people_schema))
df_parsed.printSchema()
df_parsed.show(truncate=False)

In [ ]:
df_parsed.printSchema()

In [ ]:
from pyspark.sql.functions import col
df_parsed.select(col("parsed.name"), col("parsed.payments").getItem(1)).show()

# Data Joins & Unions

## Data Join

In [ ]:
df_a = spark.createDataFrame([(1, "Alice"), (2, "Bob")],
                              ["id", "name"])
df_b = spark.createDataFrame([(3, "Charlie"), (4, "David")],
                              ["id", "name"])
result = df_a.union(df_b)
result.show()

### Union w/o duplicates

In [ ]:
df_a = spark.createDataFrame([(1, "Alice"), (2, "Bob")],
                              ["id", "name"])
df_b = spark.createDataFrame([(1, "Alice"), (4, "David")],
                              ["id", "name"])
df_a.union(df_b).distinct().show()

### Union with Mismatched Schemas

In [ ]:
df_a = spark.createDataFrame([(1, "Alice")], ["id", "name"])
df_b = spark.createDataFrame([("Bob", 2)], ["name", "id"])
result = df_a.unionByName(df_b)
result.show()

### Union by Name with missing columns

In [ ]:
df_a = spark.createDataFrame([(1, "Alice", "NY")], ["id", "name", "city"])
df_a.show()
df_b = spark.createDataFrame([(2, "Bob")], ["id", "name"])
df_b.show()
result = df_a.unionByName(df_b, allowMissingColumns=True)
result.show()

## Left Join

### Datasets

In [ ]:
book_data = [
    ("Game of thrones", 400, 1),
    ("Spark", 500, 2),
    ("Kafka", 300, 3),
    ("Java", 350, 5)
]
df_books = spark.createDataFrame(book_data, ["book_name", "cost", "writer_id"])

writer_data = [
    ("George R.R. Martin", 1),
    ("Zaharia", 2),
    ("Neha", 3),
    ("James", 4)
]
df_writers = spark.createDataFrame(writer_data, ["writer_name", "writer_id"])

df_books.show()
df_writers.show()

In [ ]:
result = df_books.join(df_writers, 
      df_books["writer_id"] == df_writers["writer_id"], "left")
result.show()

In [ ]:
result = df_books.join(df_writers, on="writer_id", how="left")
result.show()

## Right join

In [ ]:
result = df_books.join(df_writers, df_books["writer_id"] == df_writers["writer_id"], "right")
result.show()

In [ ]:
result = df_books.join(df_writers, on="writer_id", how="right")
result.show()

In [ ]:
df_writers.join(df_books, on="writer_id", how="left").show()

### Inner Join

In [ ]:
df_books.join(df_writers, on="writer_id").show()

In [ ]:
!pwd

In [ ]:
!ls /opt/spark/work-dir/data/car_service/

In [ ]:
agecnies_schema = SparkUtils.generate_schema([("agency_id", "int"), ("agency_info", "string")])
agencies_df = spark.read.option("header", True).schema(agecnies_schema).csv("/opt/spark/work-dir/data/car_service/agencies")
agencies_df.show()

In [ ]:
agencies_df = agencies_df.withColumn("agency_name", get_json_object(agencies_df.agency_info, "$.agency_name"))
agencies_df.show()

In [ ]:
sc.stop()